In [17]:
# This is the full script to produce all three models in the work in progress Tableau dashboard linked here: 
# https://public.tableau.com/app/profile/colin.mcduffie5149/viz/WebRetailerForecastingModel/MainDB
# I omitted overfit models ie. models with >.99 R2 or underfit models ie models with a low R2. 
# Generally R2 = .75 to .95 is an acceptable range but you'll have to look at the spread between the min and max of the final prediction values
# for whatever 3 models you choose. If you choose 3 models that produce a similar revenue forecast and have varied R2 in the .75 to .95 range
# then you can feel reassured that your prediction is probably accurate. This was indeed the case for our sample dataset we produced.
# in the "Web Traffic Dummy Data (3).ipynb" file. 
# If you are using this script and find that with your data, the R2 is too high or too low, please reach out to me for further assistance, or try running 
# the previous script in the "Dummy Web Revenue Forecast Model 1.ipynb" File in the github folder and omit features with low importance 
# from your dataset or try changing the date range or granularity. 

In [3]:
#You may need to install the following libraries 
pip install pandas scikit-learn statsmodels xgboost


Note: you may need to restart the kernel to use updated packages.


In [5]:
#Import Libraries

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
import xgboost as xgb
import warnings

# Optional: Suppress warnings for cleaner output. Or remove if you want to see warnings. 
warnings.filterwarnings("ignore")  

# Load the data from csv. 
df = pd.read_csv('dummy_web_traffic_revenue_data.csv')

# Data preprocessing. Convert Date field to a Date. Assign an index to the date field. Fill nulls with 0. 
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.fillna(0, inplace=True)


#Define the date range you want to use for your model. 
df = df.loc['2023-01-01':'2024-12-31']

# Drop any column with "Conversions" in its name. Or any columns you deem unnessisary. 
columns_to_drop = [col for col in df.columns if 'Conversions' in col]
df = df.drop(columns_to_drop, axis=1)

# Feature and target selection. 
#These are the features you want to use in your model. 
features = [
    'Google_Desktop_Visits', 'Google_Mobile_Visits', 'Google_Spend',
    'Facebook_Desktop_Visits', 'Facebook_Mobile_Visits', 'Facebook_Spend',
    'Direct_Desktop_Visits', 'Direct_Mobile_Visits', 'Direct_Spend', 'Bounce_Rate',
    'Avg_Session_Duration', 'Pages_Per_Session', 'Conversion_Rate', 'Total_Spend'
]

#This is the feature you want to predict. 
target = 'Total_Revenue'

# Optional: Inspect the date range in the dataset
print("Earliest date:", df.index.min())
print("Latest date:", df.index.max())

# Choose a valid split date within the date range
# Instead of a random split of 80/20 for train/test, we want to make sure to train on data before this date and test on data after this date. 
split_date = '2024-09-01'  

# Split the data into training and testing sets maintaining temporal order
train = df.loc[df.index < split_date]
test = df.loc[df.index >= split_date]

# Optional: Debug prints to check the shapes of the split datasets
print("Training set shape:", train.shape)
print("Testing set shape:", test.shape)

# Ensure the training and testing sets are not empty. If they are you'll need to move back your split date to an earlier date.  
if train.empty or test.empty:
    raise ValueError("The training or testing set is empty. Please choose an appropriate split date.")

# Standard Test/Train logic
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

# Function to evaluate model and return predictions and metrics including R2.
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"{model_name} - MAE: {mae}, MSE: {mse}, RMSE: {rmse}, R²: {r2}")
    return y_pred, mae, mse, rmse, r2

# Standard Random Forest model. 100 means run the decision tree 100 times and average the resutls. 
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions, rf_mae, rf_mse, rf_rmse, rf_r2 = evaluate_model(rf_model, X_test, y_test, "Random Forest")

# XGBoost  
xgboost_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42, n_estimators=500, learning_rate=0.05)
xgboost_model.fit(X_train, y_train)
xgboost_predictions, xgboost_mae, xgboost_mse, xgboost_rmse, xgboost_r2 = evaluate_model(xgboost_model, X_test, y_test, "XGBoost")

# SARIMAX - Good model for retailers with seasonality. 
sarimax_model = SARIMAX(y_train, exog=X_train, order=(2, 1, 2), seasonal_order=(1, 1, 1, 7), freq='D').fit(disp=False)
sarimax_predictions = sarimax_model.predict(start=test.index[0], end=test.index[-1], exog=X_test)
sarimax_mae = mean_absolute_error(y_test, sarimax_predictions)
sarimax_mse = mean_squared_error(y_test, sarimax_predictions)
sarimax_rmse = np.sqrt(sarimax_mse)
sarimax_r2 = r2_score(y_test, sarimax_predictions)
print(f"SARIMAX - MAE: {sarimax_mae}, MSE: {sarimax_mse}, RMSE: {sarimax_rmse}, R²: {sarimax_r2}")

# Compile Results into DataFrame with R2
full_actual_df = df[['Total_Revenue']].loc['2023-01-01':'2024-09-01'].copy()

# This is the format for your final table. 
results_df = pd.DataFrame({
    'Date': test.index,
    'Actual_Revenue': test[target],
    'Random_Forest_Prediction': rf_predictions,
    'XGBoost_Prediction': xgboost_predictions,
    'SARIMAX_Prediction': sarimax_predictions
})

# Adding R2 values to the DataFrame
results_df['Random_Forest_R2'] = rf_r2
results_df['XGBoost_R2'] = xgboost_r2
results_df['SARIMAX_R2'] = sarimax_r2

# Combine the full actual revenue with the results DataFrame. 
combined_df = pd.merge(full_actual_df, results_df, how='left', left_index=True, right_on='Date')
combined_df = combined_df.set_index('Date')

# Reset index to include 'Date' as a column
combined_df.reset_index(inplace=True)

# Export the DataFrame to CSV
combined_df.to_csv("final_results.csv", index=False)

print(combined_df.head(10))

Earliest date: 2023-01-01 00:00:00
Latest date: 2024-12-31 00:00:00
Training set shape: (609, 23)
Testing set shape: (122, 23)
Random Forest - MAE: 1564.2196721311475, MSE: 3860472.33442623, RMSE: 1964.8084727082764, R²: 0.8368689768843857
XGBoost - MAE: 1175.912205430328, MSE: 2193672.0168512063, RMSE: 1481.1049985909865, R²: 0.9073025450026395
SARIMAX - MAE: 776.8240832450427, MSE: 777948.9964112485, RMSE: 882.0141701873323, R²: 0.967126401972987
        Date  Total_Revenue  Actual_Revenue  Random_Forest_Prediction  \
0 2023-01-01         117780             NaN                       NaN   
1 2023-01-02         118880             NaN                       NaN   
2 2023-01-03         127180             NaN                       NaN   
3 2023-01-04         121100             NaN                       NaN   
4 2023-01-05         128260             NaN                       NaN   
5 2023-01-06         120640             NaN                       NaN   
6 2023-01-07         129620         

In [7]:
# Print the final DF 
combined_df

,Date,Total_Revenue,Actual_Revenue,Random_Forest_Prediction,XGBoost_Prediction,SARIMAX_Prediction,Random_Forest_R2,XGBoost_R2,SARIMAX_R2
0,2023-01-01,117780,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,118880,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,127180,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-04,121100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-05,128260,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
605,2024-08-28,115840,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,2024-08-29,120140,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,2024-08-30,122580,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2024-08-31,116540,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
#Output the final table to a csv. 
results_df.to_csv('dummy_web_traffic_revenue_forecast.csv', index=False)
